In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import os

In [3]:
print(os.getcwd())

/content


In [4]:
%cd /content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/NN

/content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/NN


In [5]:
from models.model_baselines import BaselineClassifier

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [7]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [8]:
%ls

'Copy of MT5_final_embeddings.ipynb'   mlp_tfidf_cv_2
'MD5_1 (1).ipynb'                      models/
 MD5_1_English.ipynb                   MT5_final_embeddings.ipynb
 mlp_tfidf_cv                          Untitled


In [9]:
train_df = pd.read_csv('../Data/Train.csv')

In [10]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [11]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [12]:
def get_model(size):
  print('Building model...')
  model = Sequential()
  model.add(Dense(2048, input_shape=(size,)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(20))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  #make one, takes in, double that amount, english and chichewa

In [13]:
def get_results(model,epochs,train_dataset,test_dataset,y_test):
  model.fit(train_dataset,
            epochs=epochs, batch_size=32,
            verbose=1)
  print(model.evaluate(test_dataset))
  ans = model.predict(test_dataset)
  print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))
  print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))

In [14]:
print(Y_train.shape)
print(train_df.shape)

(1436, 20)
(1436, 3)


In [15]:
kf = KFold(n_splits=3, random_state=42, shuffle=True)

In [16]:
for train_index, test_index in kf.split(train_df.Text):
  X_train, X_test = train_df.iloc[train_index].Text, train_df.iloc[test_index].Text
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  #print(train_index)
  #print(train_index.shape)
  #break
  tfidf = TfidfVectorizer()
  tfidf_train = tfidf.fit_transform(X_train).toarray()
  tfidf_test = tfidf.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((tfidf_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(tfidf_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
30/30 [==============================] - 2s 19ms/step - loss: 2.5245 - accuracy: 0.2153
Epoch 2/50
30/30 [==============================] - 1s 23ms/step - loss: 1.1403 - accuracy: 0.7921
Epoch 3/50
30/30 [==============================] - 1s 25ms/step - loss: 0.2571 - accuracy: 0.9728
Epoch 4/50
30/30 [==============================] - 1s 22ms/step - loss: 0.0440 - accuracy: 0.9990
Epoch 5/50
30/30 [==============================] - 1s 26ms/step - loss: 0.0157 - accuracy: 1.0000
Epoch 6/50
30/30 [==============================] - 1s 20ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 7/50
30/30 [==============================] - 1s 26ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 8/50
30/30 [==============================] - 1s 21ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 9/50
30/30 [==============================] - 1s 21ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0027 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 1s 18ms/step - loss: 2.5084 - accuracy: 0.2268
Epoch 2/50
30/30 [==============================] - 1s 18ms/step - loss: 1.0906 - accuracy: 0.8130
Epoch 3/50
30/30 [==============================] - 1s 19ms/step - loss: 0.2476 - accuracy: 0.9781
Epoch 4/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0475 - accuracy: 0.9969
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0196 - accuracy: 0.9969
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0141 - accuracy: 0.9958
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0108 - accuracy: 0.9969
Epoch 8/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0098 - accuracy: 0.9969
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0090 - accuracy: 0.9969
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0069 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 2s 27ms/step - loss: 2.5114 - accuracy: 0.2338
Epoch 2/50
30/30 [==============================] - 1s 28ms/step - loss: 1.0835 - accuracy: 0.8027
Epoch 3/50
30/30 [==============================] - 1s 27ms/step - loss: 0.2521 - accuracy: 0.9760
Epoch 4/50
30/30 [==============================] - 1s 26ms/step - loss: 0.0453 - accuracy: 0.9990
Epoch 5/50
30/30 [==============================] - 1s 24ms/step - loss: 0.0196 - accuracy: 0.9990
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0127 - accuracy: 0.9990
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0096 - accuracy: 0.9990
Epoch 8/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0067 - accuracy: 0.9990
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0065 - accuracy: 0.9990
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0036 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#f1 macro 47 43 47
#accuracy 64 60 64

In [18]:
print((47+43+47)/3)
print((64+60+64)/3)

45.666666666666664
62.666666666666664


# Count Vectorizer

In [19]:
for train_index, test_index in kf.split(train_df.Text):
  X_train, X_test = train_df.iloc[train_index].Text, train_df.iloc[test_index].Text
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  
  cv = CountVectorizer()
  cv_train = cv.fit_transform(X_train).toarray()
  cv_test = cv.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((cv_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((cv_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(cv_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
30/30 [==============================] - 1s 14ms/step - loss: 1.8834 - accuracy: 0.4974
Epoch 2/50
30/30 [==============================] - 0s 15ms/step - loss: 0.0652 - accuracy: 0.9896
Epoch 3/50
30/30 [==============================] - 0s 14ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 4/50
30/30 [==============================] - 0s 15ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 5/50
30/30 [==============================] - 0s 14ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 6/50
30/30 [==============================] - 0s 15ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 7/50
30/30 [==============================] - 0s 14ms/step - loss: 8.1218e-04 - accuracy: 1.0000
Epoch 8/50
30/30 [==============================] - 0s 14ms/step - loss: 5.8166e-04 - accuracy: 1.0000
Epoch 9/50
30/30 [==============================] - 0s 15ms/step - loss: 4.4630e-04 - accuracy: 1.0000
Epoch 10/50
30/30 [==============================] - 0s 14ms/step - loss: 3.592

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 1s 18ms/step - loss: 1.9130 - accuracy: 0.4775
Epoch 2/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0955 - accuracy: 0.9885
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0230 - accuracy: 0.9969
Epoch 4/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0152 - accuracy: 0.9958
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0123 - accuracy: 0.9958
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0166 - accuracy: 0.9969
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0089 - accuracy: 0.9969
Epoch 8/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0124 - accuracy: 0.9969
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0056 - accuracy: 0.9990
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0119 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Building model...
Epoch 1/50
30/30 [==============================] - 1s 18ms/step - loss: 1.8899 - accuracy: 0.4875
Epoch 2/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0864 - accuracy: 0.9896
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0175 - accuracy: 0.9990
Epoch 4/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0184 - accuracy: 0.9990
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0085 - accuracy: 0.9990
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0098 - accuracy: 0.9990
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0082 - accuracy: 0.9990
Epoch 8/50
30/30 [==============================] - 1s 19ms/step - loss: 0.0123 - accuracy: 0.9990
Epoch 9/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0033 - accuracy: 0.9990
Epoch 10/50
30/30 [==============================] - 1s 18ms/step - loss: 0.0109 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#f1 macro 49 43 48
#accuracy 64 58 63

In [22]:
print((49+43+48)/3)
print((64+58+63)/3)

46.666666666666664
61.666666666666664


In [29]:
for train_index, test_index in kf.split(train_df.Text):
  print(train_index[:10])

[0 1 2 3 4 5 6 7 8 9]
[ 1  4  8 10 11 13 14 15 16 17]
[ 0  2  3  5  6  7  9 10 12 15]


In [23]:
chichewa_embd = np.load('../Data/' + 'embeddings_chichewa_512_final.npy')

In [24]:
english_embd = np.load('../Data/' + 'embeddings_english_512_final.npy')

In [25]:
english_embd.shape

(1436, 512)

In [26]:
final_comb_arr = np.hstack((chichewa_embd,english_embd))

In [27]:
final_comb_arr.shape

(1436, 1024)

In [28]:
for train_index, test_index in kf.split(final_comb_arr):
  print(train_index[:10])

[0 1 2 3 4 5 6 7 8 9]
[ 1  4  8 10 11 13 14 15 16 17]
[ 0  2  3  5  6  7  9 10 12 15]


In [31]:
for train_index, test_index in kf.split(final_comb_arr):
  X_train, X_test = final_comb_arr[train_index], final_comb_arr[test_index]
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  
  #cv = CountVectorizer()
  #cv_train = cv.fit_transform(X_train).toarray()
  #cv_test = cv.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(X_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
30/30 [==============================] - 0s 3ms/step - loss: 2.5076 - accuracy: 0.2508
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 2.0880 - accuracy: 0.3908
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 1.9354 - accuracy: 0.3929
Epoch 4/50
30/30 [==============================] - 0s 3ms/step - loss: 1.7712 - accuracy: 0.4692
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 1.6542 - accuracy: 0.4953
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 1.5941 - accuracy: 0.5047
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 1.4856 - accuracy: 0.5413
Epoch 8/50
30/30 [==============================] - 0s 4ms/step - loss: 1.4242 - accuracy: 0.5705
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3483 - accuracy: 0.5925
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3270 - accuracy: 0.5893
E

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


30/30 [==============================] - 0s 3ms/step - loss: 2.4240 - accuracy: 0.2665
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 1.9681 - accuracy: 0.4190
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 1.7563 - accuracy: 0.4911
Epoch 4/50
30/30 [==============================] - 0s 3ms/step - loss: 1.6354 - accuracy: 0.5026
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 1.5226 - accuracy: 0.5413
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 1.4273 - accuracy: 0.5538
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3810 - accuracy: 0.5873
Epoch 8/50
30/30 [==============================] - 0s 3ms/step - loss: 1.3067 - accuracy: 0.5977
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 1.2426 - accuracy: 0.6113
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 1.2165 - accuracy: 0.6134
Epoch 11/50
30/30 [===========

In [ ]:
#f1 macro 45 45 45
#accuracy 54 51 58

In [32]:
print((45+45+45)/3)
print((54+51+58)/3)

45.0
54.333333333333336
